# Nama : Alisha Zahra Saadiya
# NIM : 2501971742
# Link Video : https://binusianorg-my.sharepoint.com/personal/alisha_saadiya_binus_ac_id/_layouts/15/guestaccess.aspx?share=EXNAxEUQmk1Bijw8Ql77_PwBCwA-xjonuI4AM0TiXuv8jA&nav=eyJyZWZlcnJhbEluZm8iOnsicmVmZXJyYWxBcHAiOiJPbmVEcml2ZUZvckJ1c2luZXNzIiwicmVmZXJyYWxBcHBQbGF0Zm9ybSI6IldlYiIsInJlZmVycmFsTW9kZSI6InZpZXciLCJyZWZlcnJhbFZpZXciOiJNeUZpbGVzTGlua0NvcHkifX0&e=OUK1c6

# 1. Data teks berita dengan menggunakan metode web scrapping

In [ ]:
pip install newspaper3k

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from newspaper import Article
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv('data_news.csv')
df

,URL,Media,Label
0,https://www.antaranews.com/berita/4066758/toko...,Antara News,Politik
1,https://www.antaranews.com/berita/4066710/indo...,Antara News,Politik
2,https://www.antaranews.com/berita/4066605/pilp...,Antara News,Politik
3,https://www.antaranews.com/berita/4066479/pks-...,Antara News,Politik
4,https://www.antaranews.com/berita/4066305/keju...,Antara News,Politik
...,...,...,...
95,https://mediaindonesia.com/hiburan/665400/stra...,Media Indonesia,Hiburan
96,https://mediaindonesia.com/hiburan/665050/teri...,Media Indonesia,Hiburan
97,https://mediaindonesia.com/hiburan/664944/west...,Media Indonesia,Hiburan
98,https://mediaindonesia.com/hiburan/664906/fakt...,Media Indonesia,Hiburan


In [ ]:
news = []

In [ ]:
for index, row in df.iterrows():
  article = Article(row['URL'], 'id')
  article.download()
  article.parse()

  # menghapus baris dengan awalan yg masuk pada keywords
  lines = article.text.split('\n')
  keywords = ("Baca juga:", "Copyright", "Editor:", "Pewarta:", "Baca lanjutan")
  cleaned_lines = [line for line in lines if not any(line.strip().startswith(keyword) for keyword in keywords) and "ADVERTISEMENT" not in line.strip().upper()]
  cleaned_text = '\n'.join(cleaned_lines)

  # menghapus kata yang ada di list
  words_to_remove = ["Sentani (ANTARA)","Surabaya (ANTARA)","Jakarta (ANTARA)", "Nusa Dua, Bali (ANTARA)",
                     "Semarang (ATARA)","Medan (ANTARA)", "Praya, Lombok Tengah (ANTARA)", "Denpasar (ANTARA)", "Medan (ANTARA)"]
  for word in words_to_remove:
        cleaned_text = cleaned_text.replace(word, "")

  news.append([cleaned_text, row['Media'], row['Label']])

In [ ]:
data = pd.DataFrame(news, columns=['Text', 'Media', 'Label'])

In [ ]:
data.to_csv('hasil_scrapping.csv')

In [ ]:
data.head()

,Text,Media,Label
0,Tokoh Adat Tabi harap masyarakat jaga kamtibma...,Antara News,Politik
1,Indonesia ajukan rancangan deklarasi menteri W...,Antara News,Politik
2,Pilpres 2024 perlu sandingkan Legalitas dengan...,Antara News,Politik
3,PKS sebut AS tidak mendukung terciptanya perda...,Antara News,Politik
4,Kejujuran dan bermoral jangan sampai termargin...,Antara News,Politik


# 2. Text Preprocessing (Cleansing, Tokenization, Filtering, Stemming, Lemmatization)

## Cleansing

In [ ]:
def cleansing(data):
    clean = [str(text).lower() for text in data]
    clean=[re.sub(r"\d+","",i )for i in clean]
    clean=[re.sub(r'[^\w]', ' ', i)for i in clean]
    clean=[re.sub(r'\s+',' ',i)for i in clean]
    clean = [re.sub(r'https', '', i)for i in clean]
    clean = [re.sub(r'.com', '', i)for i in clean]

    return clean

Tahapan cleansing :
- Setiap teks diubah menjadi huruf kecil dengan menggunakan method .lower()
- Menghapus angka dari setiap teks
- Mengganti tanda baca dan karakter khusus menjadi spasi
- Menghapus spasi berlebihan yang muncul setelah proses cleansing sebelumnya
- Menghapus substring https dan .com dari setiap teks


In [ ]:
data['Clean_text']=cleansing(data['Text'])

In [ ]:
data['Clean_text'].iloc[1]

'indonesia ajukan rancangan deklarasi menteri wwf ke di bali pemerintah indonesia mengajukan rancangan berisi sejumlah poin penting yang termuat dalam deklarasi tingkat menteri pada ajang world water forum forum air dunia wwf ke di nusa dua bali mei ini komitmen kami semua berkaitan dengan masalah shortage keterbatasan air di dunia kata menteri koordinator bidang kemaritiman dan investasi menko maves luhut binsar pandjaitan di sela rapat koordinasi persiapan wwf ke di nusa dua kabupaten badung bali sabtu luhut memberi sinyal deklarasi tingkat menteri itu tidak bersifat mengikat melainkan menjadi kesepakatan dan komitmen bersama terkait pentingnya ketersediaan air di dunia ia juga mengungkapkan bahwa deklarasi tingkat menteri itu juga sekaligus memuat kepentingan nasional indonesia terutama terkait air yang menjadi salah satu soal krusial dunia ke depan air menjadi masalah serius dunia itu sebabnya sangat penting kami melakukan ini serius dari pada masalah energi fosil ujarnya luhut mem

In [ ]:
data.head()

,Text,Media,Label,Clean_text
0,Tokoh Adat Tabi harap masyarakat jaga kamtibma...,Antara News,Politik,tokoh adat tabi harap masyarakat jaga kamtibma...
1,Indonesia ajukan rancangan deklarasi menteri W...,Antara News,Politik,indonesia ajukan rancangan deklarasi menteri w...
2,Pilpres 2024 perlu sandingkan Legalitas dengan...,Antara News,Politik,pilpres perlu sandingkan legalitas dengan legi...
3,PKS sebut AS tidak mendukung terciptanya perda...,Antara News,Politik,pks sebut as tidak mendukung terciptanya perda...
4,Kejujuran dan bermoral jangan sampai termargin...,Antara News,Politik,kejujuran dan bermoral jangan sampai termargin...


In [ ]:
data.to_csv('data_cleansing.csv')

In [ ]:
def label(data):
  data = data.copy()
  data['Label'] = data['Label'].apply(lambda x: 0 if x == 'Politik' else x)
  data['Label'] = data['Label'].apply(lambda x: 1 if x == 'Olahraga' else x)
  data['Label'] = data['Label'].apply(lambda x: 2 if x == 'Bisnis' else x)
  data['Label'] = data['Label'].apply(lambda x: 3 if x == 'Hiburan' else x)
  return data

In [ ]:
data = label(data)
data

,Text,Media,Label,Clean_text
0,Tokoh Adat Tabi harap masyarakat jaga kamtibma...,Antara News,0,tokoh adat tabi harap masyarakat jaga kamtibma...
1,Indonesia ajukan rancangan deklarasi menteri W...,Antara News,0,indonesia ajukan rancangan deklarasi menteri w...
2,Pilpres 2024 perlu sandingkan Legalitas dengan...,Antara News,0,pilpres perlu sandingkan legalitas dengan legi...
3,PKS sebut AS tidak mendukung terciptanya perda...,Antara News,0,pks sebut as tidak mendukung terciptanya perda...
4,Kejujuran dan bermoral jangan sampai termargin...,Antara News,0,kejujuran dan bermoral jangan sampai termargin...
...,...,...,...,...
95,Stray Kids(Instagram @realstraykids)\n\nGRUP i...,Media Indonesia,3,stray kids instagram realstraykids grup idola ...
96,Terry(MI/HO)\n\nINSPIRASI bisa datang dari man...,Media Indonesia,3,terry mi ho inspirasi bisa datang dari mana sa...
97,Westlife saat tampil di Jakarta pada 11 Februa...,Media Indonesia,3,westlife saat tampil di jakarta pada februari ...
98,Fakta dan kronologi kematian Park Boram(Instag...,Media Indonesia,3,fakta dan kronologi kematian park boram instag...


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, log_loss
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data['Clean_text'], data['Label'], test_size = 0.2, random_state = 42,stratify=data['Label'])

In [ ]:
x_train

63     kabar raffi ahmad dan nagita slavina mengadop...
16    sandiaga bayar parkir secara digital cegah pun...
49     kabid binpres pp pbsi ricky soebagdja ingin i...
92    taylor swift merilis album the tortured poets ...
17    kemenhub pantau festival balon udara wonosobo ...
                            ...                        
99    para pemeran bersama produser dan sutradara ba...
25    album baru taylor swift pecahkan sejumlah reko...
21    sulsel gencarkan program inseminasi buatan don...
4     kejujuran dan bermoral jangan sampai termargin...
79    kento momota akan pensiun pada akhir bulan ini...
Name: Clean_text, Length: 80, dtype: object

## Tokenization

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
!pip install unidecode
import unidecode

In [ ]:
x_train_stripped = [sample.strip() for sample in x_train]

x_train_unidecoded = [unidecode.unidecode(sample) for sample in x_train_stripped]

x_train_tokenized = [word_tokenize(sample) for sample in x_train_unidecoded]


In [ ]:
x_train_tokenized

[['kabar',
  'raffi',
  'ahmad',
  'dan',
  'nagita',
  'slavina',
  'mengadopsi',
  'anak',
  'perempuan',
  'santer',
  'beredar',
  'dalam',
  'beberapa',
  'hari',
  'terakhir',
  'anak',
  'perempuan',
  'yang',
  'masih',
  'bayi',
  'tersebut',
  'masih',
  'belum',
  'diungkap',
  'secara',
  'terbuka',
  'oleh',
  'raffi',
  'dan',
  'nagita',
  'kabar',
  'raffi',
  'ahmad',
  'adopsi',
  'anak',
  'pertama',
  'kali',
  'muncul',
  'saat',
  'lebaran',
  'dalam',
  'video',
  'yang',
  'diunggah',
  'pada',
  'april',
  'raffi',
  'mengajak',
  'kamera',
  'mengunjungi',
  'sebuah',
  'kamar',
  'di',
  'rumahnya',
  'di',
  'dalam',
  'kamar',
  'tersebut',
  'ada',
  'nagita',
  'slavina',
  'yang',
  'berbaring',
  'menemani',
  'seorang',
  'bayi',
  'selain',
  'nagita',
  'rafathar',
  'juga',
  'berada',
  'di',
  'samping',
  'bayi',
  'tersebut',
  'raffi',
  'kemudian',
  'bertanya',
  'kepada',
  'rafathar',
  'dan',
  'rayyanza',
  'soal',
  'bayi',
  'tersebut',

Tahapan tokenization :    
- Membersihkan teks dari spasi tambahan
- Mengonversi karakter non-ASCII menjadi ASCII, misalnya karakter é akan diubah menjadi e
- Membagi teks menjadi token (kata-kata) yang siap digunakan untuk pemrosesan lanjutan

## Filtering

In [ ]:
# Stopward removal
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('indonesian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
x_train_filtered = [[word for word in sample if word not in stop_words] for sample in x_train_tokenized]
x_train_filtered

[['kabar',
  'raffi',
  'ahmad',
  'nagita',
  'slavina',
  'mengadopsi',
  'anak',
  'perempuan',
  'santer',
  'beredar',
  'anak',
  'perempuan',
  'bayi',
  'diungkap',
  'terbuka',
  'raffi',
  'nagita',
  'kabar',
  'raffi',
  'ahmad',
  'adopsi',
  'anak',
  'kali',
  'muncul',
  'lebaran',
  'video',
  'diunggah',
  'april',
  'raffi',
  'mengajak',
  'kamera',
  'mengunjungi',
  'kamar',
  'rumahnya',
  'kamar',
  'nagita',
  'slavina',
  'berbaring',
  'menemani',
  'bayi',
  'nagita',
  'rafathar',
  'samping',
  'bayi',
  'raffi',
  'rafathar',
  'rayyanza',
  'bayi',
  'namanya',
  'percakapan',
  'keluarga',
  'bayi',
  'bernama',
  'lily',
  'berjenis',
  'kelamin',
  'perempuan',
  'gambas',
  'youtube',
  'raffi',
  'ahmad',
  'nagita',
  'slavina',
  'informasi',
  'detail',
  'bayi',
  'usulnya',
  'konfirmasi',
  'adopsi',
  'anak',
  'raffi',
  'nagita',
  'pasangan',
  'selebritas',
  'pengusaha',
  'mengunggah',
  'lily',
  'akun',
  'media',
  'sosial',
  'merah

Tahapan Filtering :
Menghapus kata-kata stopword dari teks menggunakan daftar stopword bahasa Indonesia yang disediakan oleh NLTK.

## Stemming

In [ ]:
from nltk import PorterStemmer

In [ ]:
x_train_stemmed = [[PorterStemmer().stem(word) for word in sample] for sample in x_train_filtered]
x_train_stemmed

[['kabar',
  'raffi',
  'ahmad',
  'nagita',
  'slavina',
  'mengadopsi',
  'anak',
  'perempuan',
  'santer',
  'beredar',
  'anak',
  'perempuan',
  'bayi',
  'diungkap',
  'terbuka',
  'raffi',
  'nagita',
  'kabar',
  'raffi',
  'ahmad',
  'adopsi',
  'anak',
  'kali',
  'muncul',
  'lebaran',
  'video',
  'diunggah',
  'april',
  'raffi',
  'mengajak',
  'kamera',
  'mengunjungi',
  'kamar',
  'rumahnya',
  'kamar',
  'nagita',
  'slavina',
  'berbar',
  'menemani',
  'bayi',
  'nagita',
  'rafathar',
  'samp',
  'bayi',
  'raffi',
  'rafathar',
  'rayyanza',
  'bayi',
  'namanya',
  'percakapan',
  'keluarga',
  'bayi',
  'bernama',
  'lili',
  'berjeni',
  'kelamin',
  'perempuan',
  'gamba',
  'youtub',
  'raffi',
  'ahmad',
  'nagita',
  'slavina',
  'informasi',
  'detail',
  'bayi',
  'usulnya',
  'konfirmasi',
  'adopsi',
  'anak',
  'raffi',
  'nagita',
  'pasangan',
  'selebrita',
  'pengusaha',
  'mengunggah',
  'lili',
  'akun',
  'media',
  'sosial',
  'merahasiakan',


Tahapan stemming : Menerapkan stemming pada teks yang sudah di filtering menggunakan algoritma PorterStemmer dari NLTK. Hasilnya berisi teks-teks yang telah di ubah menjadi bentuk dasar. Pada proses stemming membantu mengurangi variasi kata dalam teks sehingga dapat meningkatkan konsistensi dalam pemrosesan.

## Lemmatization

In [ ]:
from nltk import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
x_train_lemmatized = [[WordNetLemmatizer().lemmatize(word, pos='v') for word in sample] for sample in x_train_stemmed]
x_train_lemmatized

[['kabar',
  'raffi',
  'ahmad',
  'nagita',
  'slavina',
  'mengadopsi',
  'anak',
  'perempuan',
  'santer',
  'beredar',
  'anak',
  'perempuan',
  'bayi',
  'diungkap',
  'terbuka',
  'raffi',
  'nagita',
  'kabar',
  'raffi',
  'ahmad',
  'adopsi',
  'anak',
  'kali',
  'muncul',
  'lebaran',
  'video',
  'diunggah',
  'april',
  'raffi',
  'mengajak',
  'kamera',
  'mengunjungi',
  'kamar',
  'rumahnya',
  'kamar',
  'nagita',
  'slavina',
  'berbar',
  'menemani',
  'bayi',
  'nagita',
  'rafathar',
  'samp',
  'bayi',
  'raffi',
  'rafathar',
  'rayyanza',
  'bayi',
  'namanya',
  'percakapan',
  'keluarga',
  'bayi',
  'bernama',
  'lili',
  'berjeni',
  'kelamin',
  'perempuan',
  'gamba',
  'youtub',
  'raffi',
  'ahmad',
  'nagita',
  'slavina',
  'informasi',
  'detail',
  'bayi',
  'usulnya',
  'konfirmasi',
  'adopsi',
  'anak',
  'raffi',
  'nagita',
  'pasangan',
  'selebrita',
  'pengusaha',
  'mengunggah',
  'lili',
  'akun',
  'media',
  'sosial',
  'merahasiakan',


In [ ]:
x_train_processed = [' '.join(sample) for sample in x_train_lemmatized]

x_train_processed

['kabar raffi ahmad nagita slavina mengadopsi anak perempuan santer beredar anak perempuan bayi diungkap terbuka raffi nagita kabar raffi ahmad adopsi anak kali muncul lebaran video diunggah april raffi mengajak kamera mengunjungi kamar rumahnya kamar nagita slavina berbar menemani bayi nagita rafathar samp bayi raffi rafathar rayyanza bayi namanya percakapan keluarga bayi bernama lili berjeni kelamin perempuan gamba youtub raffi ahmad nagita slavina informasi detail bayi usulnya konfirmasi adopsi anak raffi nagita pasangan selebrita pengusaha mengunggah lili akun media sosial merahasiakan rupa bayi raffi nagita mengonfirmasi nama bayi lili menganggap anak perempuan malaikat dikirim allah swt gamba instagram gamba instagram gamba instagram gamba instagram gamba instagram gamba instagram lantaran identita bayi perempuan netizen menduga duga muasal lili salah santer beredar lili anak palestina diadopsi raffi ahmad nagita slavina raffi ahmad menikah nagita slavina oktob agustu dikaruniai 

Tahapan Lemmatization:
- Menerapkan lemmatisasi pada teks stemming menggunakan algoritma WordNetLemmatizer dari NLTK. Hasilnya berisi teks yang diubah menjadi bentuk dasar sebenarnya
- Mengubah teks yang telah di lematisasi kembali menjadi string yang lengkap

# Metode TFIDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
TFIDF_vectorizer = TfidfVectorizer()
train_tfidf = TFIDF_vectorizer.fit_transform(x_train)

In [ ]:
TFIDF_train = pd.DataFrame(train_tfidf.toarray(), columns=TFIDF_vectorizer.get_feature_names_out())
TFIDF_train.head()

,_everything,abah,abdel,abdullah,absen,abstain,abu,acara,acc,access,...,zaki,zaman,zedd,zero,zhou,zita,zona,zuhro,zulhas,zulkifli
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023930,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.031728,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
test_tfidf = TFIDF_vectorizer.transform(x_test)

In [ ]:
TFIDF_test = pd.DataFrame(test_tfidf.toarray(), columns=TFIDF_vectorizer.get_feature_names_out())
TFIDF_test.head()

,_everything,abah,abdel,abdullah,absen,abstain,abu,acara,acc,access,...,zaki,zaman,zedd,zero,zhou,zita,zona,zuhro,zulhas,zulkifli
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.036799,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Tahapan TF-IDF  :
1. Membuat objek TfidfVectorizer yang akan digunakan untuk mengonversi teks menjadi representasi TF-IDF. TFIDFVectorizer digunakan untuk mengubah dokumen teks menjadi matriks yang berisi bobot TF-IDF dari setiap kata di setiap dokumen.
2. Memanggil metode fit_transform pada objek TfidfVectorizer untuk mempelajari vektorisasi dari data x_train maupun x_test. Metode ini melakukan dua hal:
- Pertama, mempelajari kosakata dari data x_train maupun x_test dan menghitung bobot IDF (Inverse Document Frequency) untuk setiap kata di dalamnya.
- Kedua, mentransformasikan data latih menjadi matriks TF-IDF yang merupakan representasi vektor dari setiap dokumen.
3. Membangun sebuah data frame dari array matriks TF-IDF dan di representasikan menjadi kolom-kolom yang mewakili fitur-fitur kata dalam teks.


## SVM

In [ ]:
from sklearn import svm
SVM_TFIDF = svm.LinearSVC(penalty='l2',
                          loss='squared_hinge',
                          dual=True,
                          verbose=0,
                          random_state=42)
SVM_TFIDF.fit(TFIDF_train, y_train)

LinearSVC(random_state=42)

In [ ]:
predict_SVM_TFIDF = SVM_TFIDF.predict(TFIDF_test)

In [ ]:
print(classification_report(y_test.reset_index(drop=True), predict_SVM_TFIDF, target_names=['0','1','2','3']))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00         5
           2       0.83      1.00      0.91         5
           3       1.00      0.80      0.89         5

    accuracy                           0.95        20
   macro avg       0.96      0.95      0.95        20
weighted avg       0.96      0.95      0.95        20



In [ ]:
# SVM setelah tuning parameter
SVM_TFIDF2 = svm.LinearSVC(
                           tol=0.0001,
                           C=1.0,
                           intercept_scaling=1,
                           verbose=0,
                           random_state=None,
                           max_iter=1000)

# Melatih model dengan data TF-IDF train
SVM_TFIDF2.fit(TFIDF_train, y_train)

LinearSVC()

In [ ]:
# Prediksi menggunakan model LinearSVC yang telah diubah parameter
predict_SVM_TFIDF2 = SVM_TFIDF2.predict(TFIDF_test)

In [ ]:
# Evaluasi model SVM TF-IDF setelah diubah parameter menggunakan classification report
print(classification_report(y_test, predict_SVM_TFIDF2, target_names=['0','1','2','3']))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00         5
           2       0.83      1.00      0.91         5
           3       1.00      0.80      0.89         5

    accuracy                           0.95        20
   macro avg       0.96      0.95      0.95        20
weighted avg       0.96      0.95      0.95        20



## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RF_TFIDF = RandomForestClassifier(n_estimators=200,
    criterion='entropy',
    random_state=42)
RF_TFIDF.fit(TFIDF_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=200, random_state=42)

In [ ]:
predict_RF_TFIDF = RF_TFIDF.predict(TFIDF_test)

In [ ]:
print(classification_report(y_test, predict_RF_TFIDF, target_names=['0','1','2','3']))

              precision    recall  f1-score   support

           0       0.75      0.60      0.67         5
           1       1.00      1.00      1.00         5
           2       0.80      0.80      0.80         5
           3       0.67      0.80      0.73         5

    accuracy                           0.80        20
   macro avg       0.80      0.80      0.80        20
weighted avg       0.80      0.80      0.80        20



In [ ]:
# Random Forest setelah tuning parameter
RF_TFIDF2= RandomForestClassifier(n_estimators=100,
                                  criterion='gini',
                                  max_features='sqrt',
                                  random_state=None,
                                  max_samples=None)

# Melatih model dengan data TF-IDF train
RF_TFIDF2.fit(TFIDF_train, y_train)

# Prediksi menggunakan model RandomForestClassifier yang telah diubah parameter
predict_RF_TFIDF2 = RF_TFIDF2.predict(TFIDF_test)

In [ ]:
# Evaluasi model RF TF-IDF setelah diubah parameter menggunakan classification report
print(classification_report(y_test, predict_RF_TFIDF2, target_names=['0','1','2','3']))

              precision    recall  f1-score   support

           0       1.00      0.80      0.89         5
           1       1.00      1.00      1.00         5
           2       0.83      1.00      0.91         5
           3       0.80      0.80      0.80         5

    accuracy                           0.90        20
   macro avg       0.91      0.90      0.90        20
weighted avg       0.91      0.90      0.90        20



# Metode Word2Vec - Skipgram

In [ ]:
import gensim
from gensim.models import Word2Vec

In [ ]:
x_train_list = [text.split() for text in x_train]

In [ ]:
model_W2V = gensim.models.Word2Vec(x_train_list, min_count = 3,vector_size = 50, sg = 1)

In [ ]:
vocabulary_W2V = model_W2V.wv.index_to_key
print(vocabulary_W2V)

['yang', 'di', 'dan', 'untuk', 'dengan', 'ini', 'dalam', 'pada', 'dari', 'itu', 'juga', 'indonesia', 'akan', 'tidak', 'ke', 'tahun', 'tersebut', 'menjadi', 'ada', 'bisa', 'sudah', 'jakarta', 'saat', 'dia', 'karena', 'saya', 'kata', 'kita', 'sebagai', 'satu', 'oleh', 'mereka', 'lebih', 'telah', 'ia', 'bahwa', 'banyak', 'kami', 'dapat', 'baca', 'tim', 'atau', 'sangat', 'kepada', 'baik', 'seperti', 'setelah', 'hari', 'jadi', 'bagi', 'pemilu', 'adalah', 'baru', 'atlet', 'terhadap', 'masyarakat', 'namun', 'antara', 'tapi', 'merupakan', 'the', 'melalui', 'ketua', 'dki', 'mengatakan', 'bersama', 'pertama', 'rp', 'atas', 'hal', 'kembali', 'melakukan', 'orang', 'sama', 'terkait', 'harus', 'secara', 'hasil', 'salah', 'negara', 'serta', 'tetap', 'para', 'terus', 'hingga', 'kementerian', 'ekonomi', 'lalu', 'belum', 'album', 'april', 'memberikan', 'lain', 'anak', 'red', 'lagu', 'kedua', 'dpr', 'selain', 'waktu', 'beberapa', 'film', 'hukum', 'memiliki', 'umum', 'asia', 'mendukung', 'masih', 'besar',

In [ ]:
vectors_W2V = [model_W2V.wv.get_vector(word)for word in vocabulary_W2V]
vectors_W2V

[array([-0.12559184,  0.2703024 , -0.03818901, -0.05909735,  0.00816775,
        -0.37558687,  0.33384755,  0.25602293, -0.28060758, -0.25271255,
         0.16271242, -0.07880003, -0.12997581,  0.24939267,  0.00058101,
         0.05791085,  0.06944137, -0.08700796, -0.31332955, -0.27330297,
         0.07996516,  0.28808782,  0.26273677,  0.04949174,  0.11733623,
         0.04207354, -0.22869866, -0.11666977, -0.13883798, -0.06172442,
        -0.08685113, -0.2712509 , -0.14870217,  0.01821491, -0.22743638,
         0.13440575,  0.2667233 , -0.15905818, -0.10227804, -0.16290052,
         0.46382567,  0.04126519, -0.00659484, -0.01789441,  0.44261414,
         0.1305773 , -0.05166566, -0.02079268,  0.22665052,  0.20219514],
       dtype=float32),
 array([-0.13129644,  0.2861417 , -0.05258756, -0.10905566,  0.02025837,
        -0.38925287,  0.33109143,  0.22155099, -0.27470094, -0.2106237 ,
         0.11887974, -0.06266363, -0.17455687,  0.18355097,  0.00302904,
         0.08559025,  0.062

In [ ]:
def W2V_vectorizer(text, model_W2V):
    words = text.split()
    vectors = []
    for word in words:
        if word in model_W2V.wv:
            vectors.append(model_W2V.wv[word])
    if not vectors:
        return np.zeros(100)
    return np.mean(vectors, axis=0)

W2V_train = np.array([W2V_vectorizer(text, model_W2V) for text in x_train])

In [ ]:
W2V_train

array([[-0.11917716,  0.25295162, -0.04704872, ..., -0.06076845,
         0.18924403,  0.16261981],
       [-0.11157791,  0.24390651, -0.04703688, ..., -0.05414666,
         0.19399656,  0.16772567],
       [-0.11428954,  0.24148639, -0.04798609, ..., -0.06088137,
         0.20224135,  0.17206044],
       ...,
       [-0.11096508,  0.23892541, -0.04716552, ..., -0.05626476,
         0.1872561 ,  0.16011332],
       [-0.1130897 ,  0.24930295, -0.04730448, ..., -0.05932522,
         0.19281499,  0.16479583],
       [-0.11847936,  0.24648653, -0.04725722, ..., -0.0596253 ,
         0.19936706,  0.16819008]], dtype=float32)

In [ ]:
W2V_test = np.array([W2V_vectorizer(text, model_W2V) for text in x_test])
W2V_test

array([[-1.17058285e-01,  2.49155506e-01, -4.85473722e-02,
        -7.86928833e-02,  6.75655715e-03, -3.68170619e-01,
         3.17108870e-01,  2.43589088e-01, -2.68067062e-01,
        -2.26483449e-01,  1.34881228e-01, -6.71923459e-02,
        -1.33694381e-01,  2.15473443e-01, -6.00699475e-03,
         7.96249881e-02,  7.27315173e-02, -8.31828266e-02,
        -3.08253139e-01, -2.53138691e-01,  5.89834116e-02,
         2.71711737e-01,  2.62197912e-01,  3.08292247e-02,
         8.95437971e-02,  4.84420545e-02, -2.29804575e-01,
        -1.39710546e-01, -1.37490466e-01, -3.34331281e-02,
        -5.16902953e-02, -2.51645207e-01, -1.71072453e-01,
         5.65503389e-02, -2.18076363e-01,  1.49896905e-01,
         2.16543838e-01, -1.62183300e-01, -9.59329680e-02,
        -1.55908957e-01,  4.88900244e-01,  3.29459012e-02,
         2.78046802e-02, -5.41605428e-03,  4.20872211e-01,
         1.39008075e-01, -3.98251154e-02, -5.83482087e-02,
         2.00905889e-01,  1.70447454e-01],
       [-1.18

Tahapan W2V - Skipgram :    
1. Membuat list baru yang berisi data dari x_train yang telah dibagi menjadi kata-kat.
2. Membuat model Word2Vec menggunakan library Gensin. Untuk menunjukkan penggunaan skipgram maka isi sg  = 1.
3.  Mengambil kumpulan kata yang terdapat dalam model W2V dan disimpan dalam suatu variabel
4. Membuat variabel untuk menghasilkan vektor kata untuk setiap kata dalam variabel vocabulary_W2V dan menyimpannya
5. Membuat fungsi untuk menghasilkan representasi vektor W2V untuk teks yang diberikan
6. Membuat representasi vektor W2V untuk data x_train dan x_test menggunakan fungsi yang sudah dibuat

## SVM

In [ ]:
SVM_W2V = svm.LinearSVC(
    penalty='l2',
    loss='hinge',
    tol=0.0001,
    C=0.1,
    random_state=42,
    max_iter=1000
)

# Melatih model dengan data W2V train
SVM_W2V.fit(W2V_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(C=0.1, loss='hinge', random_state=42)

In [ ]:
# Prediksi menggunakan model LinearSVC
predict_SVM_W2V = SVM_W2V.predict(W2V_test)

In [ ]:
# Evaluasi model menggunakan classification report
print(classification_report(y_test, predict_SVM_W2V, target_names=['0','1','2','3']))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.00      0.00      0.00         5
           2       0.00      0.00      0.00         5
           3       0.25      1.00      0.40         5

    accuracy                           0.25        20
   macro avg       0.06      0.25      0.10        20
weighted avg       0.06      0.25      0.10        20



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# SVM setelah tuning parameter
SVM_W2V2 = svm.LinearSVC(
    tol=0.001,
    C=0.01,
    multi_class='ovr',
    fit_intercept=False,
    class_weight='balanced',
    random_state=None,
    max_iter=2000
)

# Melatih model dengan data W2V train
SVM_W2V2.fit(W2V_train, y_train)

LinearSVC(C=0.01, class_weight='balanced', fit_intercept=False, max_iter=2000,
          tol=0.001)

In [ ]:
# Prediksi menggunakan model LinearSVC
predict_SVM_W2V2 = SVM_W2V2.predict(W2V_test)

In [ ]:
# Evaluasi model menggunakan classification report
print(classification_report(y_test, predict_SVM_W2V2, target_names=['0','1','2','3']))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.00      0.00      0.00         5
           2       0.00      0.00      0.00         5
           3       0.25      1.00      0.40         5

    accuracy                           0.25        20
   macro avg       0.06      0.25      0.10        20
weighted avg       0.06      0.25      0.10        20



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Random Forest

In [ ]:
RF_W2V= RandomForestClassifier(n_estimators=200,
                                criterion='entropy',
                                max_features='log2',
                                random_state=42,
                                class_weight='balanced',
                                ccp_alpha=0.1)

# Melatih model dengan data W2V train
RF_W2V.fit(W2V_train, y_train)

# Prediksi menggunakan model RandomForestClassifier
predict_RF_W2V = RF_W2V.predict(W2V_test)

In [ ]:
# Evaluasi model RF W2V menggunakan classification report
print(classification_report(y_test, predict_RF_W2V, target_names=['0','1','2','3']))

              precision    recall  f1-score   support

           0       1.00      0.60      0.75         5
           1       0.43      0.60      0.50         5
           2       0.60      0.60      0.60         5
           3       0.60      0.60      0.60         5

    accuracy                           0.60        20
   macro avg       0.66      0.60      0.61        20
weighted avg       0.66      0.60      0.61        20



In [ ]:
# Random Forest setelah tuning parameter
RF_W2V2= RandomForestClassifier(
                                  criterion='gini',
                                  max_depth=15,
                                  min_samples_leaf=5,
                                  max_features='auto',
                                  random_state=None,
                                  class_weight='balanced')

# Melatih model dengan data W2V train
RF_W2V2.fit(W2V_train, y_train)

# Prediksi menggunakan model RandomForestClassifier yang telah diubah parameter
predict_RF_W2V2 = RF_W2V2.predict(W2V_test)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


In [ ]:
# Evaluasi model RF TF-IDF setelah diubah parameter menggunakan classification report
print(classification_report(y_test, predict_RF_W2V2, target_names=['0','1','2','3']))

              precision    recall  f1-score   support

           0       1.00      0.40      0.57         5
           1       0.50      0.80      0.62         5
           2       0.67      0.80      0.73         5
           3       0.75      0.60      0.67         5

    accuracy                           0.65        20
   macro avg       0.73      0.65      0.65        20
weighted avg       0.73      0.65      0.65        20

